# Introduction

In this notebook, we will show you how to work with a relational database using Python, and a SQL Connector. Connectors are typically provided by the database developer to help programmers write their own programs using that database. They may also be created by third-party developers to help with specific kinds of uses for the database. 

First, let's open our CSV (`Calgary_Public_Library_Locations_and_Hours.csv`) in pandas. 

This CSV is provided courtesy of the City of Calgary, under the Open Government License, and can be found here: https://data.calgary.ca/Recreation-and-Culture/Calgary-Public-Library-Locations-and-Hours/m9y7-ui7j

In [1]:

import pandas as pd

cpl_locations = pd.read_csv("Calgary_Public_Library_Locations_and_Hours.csv")
cpl_locations.head()
print(cpl_locations.columns.tolist())


['Library', 'Postal_Code', 'Square_Feet', 'Phone_Number', 'Monday_Open', 'Monday_Close', 'Tuesday_Open', 'Tuesday_Close', 'Wednesday_Open', 'Wednesday_Close', 'Thursday_Open', 'Thursday_Close', 'Friday_Open', 'Friday_Close', 'Saturday_Open', 'Saturday_Close', 'Sunday_Open', 'Sunday_Close', 'Address']


# Using Python to create a table

First, we're going to create your table using the [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/).

Always have the documentation for your connector available, as different implementations of database connectors may vary in terms of what methods are available and how they are used.

Although we are running MariaDB on the server, we are using the connector from MySQL on the _client side_ as it is compatible with MariaDB.

In [2]:
import mysql.connector
from mysql.connector import errorcode

# uncomment the lines below and fill in any relevant details that need to be changed here, such as if you set up a different user or password


myconnection = mysql.connector.connect(user='zheyu_song', password='5QFH73NUI',
                                 host='datasciencedb.ucalgary.ca', database='zheyu_song')
# 

In [3]:
# CREATE TABLE STATEMENT
create_statement = '''create table library_locations (
    Library varchar(40) NOT NULL,
    Postal_Code varchar(7),
    Square_Feet int, 
    Phone_Number varchar(12),
    Monday_Open time,
    Monday_Close time,
    Tuesday_Open time,
    Tuesday_Close time,
    Wednesday_Open time,
    Wednesday_Close time,
    Thursday_Open time,
    Thursday_Close time,
    Friday_Open time,
    Friday_Close time,
    Saturday_Open time,
    Saturday_Close time,
    Sunday_Open time,
    Sunday_Close time,
    Address  varchar(100)
    );'''

# now we'll create a cursor and run our create statement
create_cursor = myconnection.cursor()
try:
    create_cursor.execute(create_statement)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("Ooops! We already have that table")
    else:
        print(err.msg)
else:
    print("table created successfully!")

create_cursor.close()

table created successfully!


True

# Using Python to insert data

Because we have already built a dataframe with data from our CSV, we are going to use this datafrarame by reading each line, and writing each line into its own insert command.

How would you re-write the block of code below to make it more efficient?

In [4]:
insertCursor = myconnection.cursor()

columnString = "`,`".join([str(currentColumn) for currentColumn in cpl_locations.columns.tolist()])
#print (columnString)

# inserting rows one by one from the DataFrame is sufficient for now
for i, currentRow in cpl_locations.iterrows():
    print (tuple(currentRow))
    insertCommand = "INSERT INTO `library_locations` (`" + columnString + "`) VALUES (" + "%s,"*(len(currentRow)-1) + "%s)"
    print (insertCommand)
    insertCursor.execute(insertCommand, tuple(currentRow))
    
myconnection.commit()

insertCursor.close()

('W.R. Castell Central Library', 'T2G 2M2', 177532, '403-260-2600', '9:00', '20:00', '9:00', '20:00', '9:00', '20:00', '9:00', '20:00', '9:00', '17:00', '10:00', '17:00', '12:00', '17:00', '616 Macleod Tr SE\n(51.0470276, -114.0578995)')
INSERT INTO `library_locations` (`Library`,`Postal_Code`,`Square_Feet`,`Phone_Number`,`Monday_Open`,`Monday_Close`,`Tuesday_Open`,`Tuesday_Close`,`Wednesday_Open`,`Wednesday_Close`,`Thursday_Open`,`Thursday_Close`,`Friday_Open`,`Friday_Close`,`Saturday_Open`,`Saturday_Close`,`Sunday_Open`,`Sunday_Close`,`Address`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
('Alexander Calhoun Library', 'T2T 3V8', 9256, '403-260-2600', '10:00', '21:00', '10:00', '21:00', '10:00', '21:00', '10:00', '21:00', '10:00', '18:00', '10:00', '17:00', '12:00', '17:00', '3223 14 St SW\n(51.0255318, -114.0947876)')
INSERT INTO `library_locations` (`Library`,`Postal_Code`,`Square_Feet`,`Phone_Number`,`Monday_Open`,`Monday_Close`,`Tuesday_Open`,`Tuesday_Close`,

True

# Using Python to retrieve data

You can also use cursors to read data from a database table. It is helpful to specify what kind of result set you would like the cursor to return. Try each of the following for the second arguement:
* `raw=True`
* `dictionary=True`
* `named_tuple=True`

In [5]:
# try changing the second argument in this method call.
read_cursor = myconnection.cursor(buffered=True, named_tuple=True)

query_string = ("SELECT library FROM library_locations;")

read_cursor.execute(query_string)

for (library_value) in read_cursor:
    print(library_value)
    
read_cursor.close()

Row(library='W.R. Castell Central Library')
Row(library='Alexander Calhoun Library')
Row(library='Bowness Library')
Row(library='Fish Creek Library')
Row(library='Forest Lawn Library')
Row(library='Glenmore Square Library')
Row(library='Louise Riley Library')
Row(library='Memorial Park Library')
Row(library='Nose Hill Library')
Row(library='Shawnessy Library')
Row(library='Signal Hill Library')
Row(library='Southwood Library')
Row(library='Judith Umbach Library')
Row(library='Village Square Library')
Row(library='Crowfoot Library')
Row(library='Country Hills Library')
Row(library='Saddletowne Library')
Row(library='Westbrook Library')


True

# Parameters in SQL

_Parameterization_ is an important feature of cursors which let you create (or *prepare*) a general statement which you can "fill in" with specific values later. This is a more efficient way to use a database rather than submit specific queries.

For the MySQL Connector, the %s symbol is usually used to indicate when a parameter should be used. 

_Can you see where in this notebook we have already used this feature in a statement?_

In [6]:
import datetime

size_cursor = myconnection.cursor(buffered=True, dictionary=True)
library_sizes = [0, 10000, 25000, 100000]

query_string = ("SELECT library FROM library_locations where square_feet < %s ;")
for current_size in library_sizes:
    print ("Number of libraries with square feet less than", current_size)
    size_cursor.execute(query_string, (current_size,))
    print (size_cursor.rowcount)

size_cursor.close()

Number of libraries with square feet less than 0
0
Number of libraries with square feet less than 10000
6
Number of libraries with square feet less than 25000
15
Number of libraries with square feet less than 100000
17


True

In [7]:
# CLEANUP: always remember to release the resources you have used on the server. Always run this cell last!

myconnection.close()